In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
transactions = pd.read_csv('transactions.csv')
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')

Mission n° 1
Avant de pouvoir entrer dans le vif du sujet, il vous faudra faire un peu de nettoyage ! Par exemple, vous devrez faire des choix quant au traitement des valeurs manquantes et des valeurs aberrantes.

1 - TRANSACTIONS

In [3]:
transactions


,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [4]:
transactions.describe(include = 'all')

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB


In [6]:
#Dénombrement des valeurs manquantes
transactions.isna().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

In [7]:
#Il y a des valeurs "test_***" à supprimer car certaines sont en doubles et ce sont des essais. 
#On les retrouves toutes sous "session_id" = s_O.
pd.set_option('mode.chained_assignment', None)
transactions = transactions.drop_duplicates()
transactions

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [8]:
#Affichage des données associées aux sessions de test
transactions.loc[transactions['session_id'] == 's_0']

,id_prod,date,session_id,client_id
1431,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1
2365,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_1
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1
5955,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0
7283,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_1
...,...,...,...,...
264317,T_0,test_2021-03-01 02:30:02.237416,s_0,ct_1
288918,T_0,test_2021-03-01 02:30:02.237415,s_0,ct_1
293107,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_0
298399,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1


In [9]:
index_s_0 = transactions[transactions['session_id'] == 's_0'].index.values
transactions.drop(index_s_0, 0, inplace=True)
transactions.loc[transactions['session_id'] == 's_0']

,id_prod,date,session_id,client_id


Analyse des types

In [10]:
print(transactions.dtypes)


id_prod       object
date          object
session_id    object
client_id     object
dtype: object


In [11]:
#Toutes les variables sont "object" mais "date" peut-être transformé en "datetime"
transactions['date'] = pd.to_datetime(transactions['date'])
print(transactions.dtypes)

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object


In [12]:
#Cela permet de classer facile la table par ordre chronologique
transactions = transactions.sort_values('date')
transactions = transactions.set_index('date')
print(transactions.head())

                           id_prod session_id client_id
date                                                   
2021-03-01 00:01:07.843138  0_1259        s_1     c_329
2021-03-01 00:02:26.047414  0_1390        s_2     c_664
2021-03-01 00:02:38.311413  0_1352        s_3     c_580
2021-03-01 00:04:54.559692  0_1458        s_4    c_7912
2021-03-01 00:05:18.801198  0_1358        s_5    c_2033


2 - PRODUCTS

In [13]:
products

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
...,...,...,...
3282,2_23,115.99,2
3283,0_146,17.14,0
3284,0_802,11.22,0
3285,1_140,38.56,1


In [14]:
products.dtypes

id_prod     object
price      float64
categ        int64
dtype: object

In [15]:
#On peut modifier "categ" en variable catégorielle, et classer la table par catégorie
products['categ'] = products['categ'].astype('category')
products.dtypes

products = products.sort_values('categ')
products

,id_prod,price,categ
0,0_1421,19.99,0
1985,0_724,21.78,0
1986,0_1467,4.99,0
1988,0_1076,25.11,0
1989,0_2211,9.99,0
...,...,...,...
719,2_223,175.99,2
1272,2_7,69.99,2
1729,2_84,182.30,2
1284,2_211,203.99,2


In [16]:
#Vérification de la présence de valeurs manquantes
products.isna().sum()

id_prod    0
price      0
categ      0
dtype: int64

In [17]:
#Vérification de la présence de doublons
duplicated = products.duplicated()
products[duplicated]

,id_prod,price,categ


In [18]:
products_test = products.sort_values('price', ascending=True)
products_test

,id_prod,price,categ
731,T_0,-1.00,0
2272,0_528,0.62,0
2355,0_202,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0
...,...,...,...
2779,2_30,233.54,2
1436,2_167,236.99,2
394,2_158,247.22,2
724,2_76,254.44,2


In [19]:
#On supprime les id_prod "T_0", comme fait précédemment dans le fichier transactions, car ils correspondent à des tests
#et son prix est de -1 EUR
products.drop(731, 0, inplace=True)
products.loc[products['id_prod'] == 'T_0']

,id_prod,price,categ


3 - CLIENTS

In [20]:
customers

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


In [21]:
print(customers.dtypes)

client_id    object
sex          object
birth         int64
dtype: object


In [22]:
#La variable "sex" peut être passée en catégorielle
customers['sex'] = customers['sex'].astype('category')
customers.dtypes

client_id      object
sex          category
birth           int64
dtype: object

In [23]:
#Recherche de valeurs manquantes
customers.isna().sum()

client_id    0
sex          0
birth        0
dtype: int64

In [24]:
#Recherche de données dupliquées
duplicated_customers = customers.duplicated()
customers[duplicated_customers]

,client_id,sex,birth


In [25]:
#Suppression des clients (ct_0 et ct_1) associés aux sessions de test
ct_0 = customers.loc[customers['client_id'] == 'ct_0']
ct_1 = customers.loc[customers['client_id'] == 'ct_1']
print(ct_0)
print(ct_1)

     client_id sex  birth
2735      ct_0   f   2001
     client_id sex  birth
8494      ct_1   m   2001


In [26]:
customers.drop([2735, 8494], 0, inplace=True)
customers.sort_values('client_id')

,client_id,sex,birth
4299,c_1,m,1955
6894,c_10,m,1956
2137,c_100,m,1992
8472,c_1000,f,1966
3426,c_1001,m,1982
...,...,...,...
7004,c_995,m,1955
2788,c_996,f,1970
94,c_997,f,1994
2145,c_998,m,2001


4 - DATAFRAMES

In [27]:
#Si des id_prod sont présents dans transactions mais pas dans products, alors on doit les supprimer car ils ne seront 
#pas associés à un prix dans products
transactions['id_prod_prod'] = transactions['id_prod'].isin(products['id_prod'])
transactions['client_id_custom'] = transactions['client_id'].isin(customers['client_id'])
transactions

,id_prod,session_id,client_id,id_prod_prod,client_id_custom
date,,,,,
2021-03-01 00:01:07.843138,0_1259,s_1,c_329,True,True
2021-03-01 00:02:26.047414,0_1390,s_2,c_664,True,True
2021-03-01 00:02:38.311413,0_1352,s_3,c_580,True,True
2021-03-01 00:04:54.559692,0_1458,s_4,c_7912,True,True
2021-03-01 00:05:18.801198,0_1358,s_5,c_2033,True,True
...,...,...,...,...,...
2022-02-28 23:56:57.196281,1_456,s_172422,c_5787,True,True
2022-02-28 23:56:57.561424,1_370,s_172423,c_1460,True,True
2022-02-28 23:57:12.741573,0_1538,s_172424,c_1876,True,True


In [28]:
#Si id_prod_prod = False, alors id_prod est uniquement dans transactions
id_prod_false = transactions[transactions['id_prod_prod'] == False]
id_prod_false = id_prod_false.groupby('id_prod').mean()
id_prod_false

,id_prod_prod,client_id_custom
id_prod,,
0_2245,False,True


In [29]:
products_0_2245 = products.loc[products['id_prod'] == '0_2245']
print(products_0_2245)

Empty DataFrame
Columns: [id_prod, price, categ]
Index: []


In [30]:
#0_2245 n'est donc pas présent dans produits mais dans transactions. Or il a été acheté 103 fois et il faudrait l'inclure 
#dans l'analyse. On peut donc lui attribuer, comme prix, la moyenne des prix de sa catégorie
transactions_m = pd.merge(transactions, products, on=['id_prod']) 
transactions_m = pd.pivot_table(index='id_prod', columns='categ', values='price', aggfunc=np.mean, data=transactions_m)
moy_cat0 = transactions_m[0].mean(skipna=True)
moy_cat0

11.718568310781567

In [31]:
#On attribue ce nouveau prix à 0_2245 dans products
products = products.sort_index()
print(products.tail())

     id_prod   price categ
3282    2_23  115.99     2
3283   0_146   17.14     0
3284   0_802   11.22     0
3285   1_140   38.56     1
3286  0_1920   25.16     0


In [32]:
products.loc[3287] = {'id_prod' : '0_2245', 'price' : 11.72, 'categ' : 0}
print(products.tail())

     id_prod  price  categ
3283   0_146  17.14      0
3284   0_802  11.22      0
3285   1_140  38.56      1
3286  0_1920  25.16      0
3287  0_2245  11.72      0


In [33]:
transactions

,id_prod,session_id,client_id,id_prod_prod,client_id_custom
date,,,,,
2021-03-01 00:01:07.843138,0_1259,s_1,c_329,True,True
2021-03-01 00:02:26.047414,0_1390,s_2,c_664,True,True
2021-03-01 00:02:38.311413,0_1352,s_3,c_580,True,True
2021-03-01 00:04:54.559692,0_1458,s_4,c_7912,True,True
2021-03-01 00:05:18.801198,0_1358,s_5,c_2033,True,True
...,...,...,...,...,...
2022-02-28 23:56:57.196281,1_456,s_172422,c_5787,True,True
2022-02-28 23:56:57.561424,1_370,s_172423,c_1460,True,True
2022-02-28 23:57:12.741573,0_1538,s_172424,c_1876,True,True


In [34]:
df = pd.merge(transactions, customers, on='client_id')
df = pd.merge(df, products, on = 'id_prod')
df['count'] = 1
df = df.groupby('client_id').sum().reset_index()
df = df.sort_values('count', ascending=False)
df = df[['client_id', 'count']]
df = pd.merge(df, customers, on='client_id')
df = df.sort_values('count', ascending=False)
top_10 = df.iloc[0:10]
print(top_10)

   client_id  count sex  birth
0     c_1609  12855   m   1980
1     c_6714   4473   f   1968
2     c_3454   3275   m   1969
3     c_4958   2562   m   1999
4     c_2140    195   f   1977
5     c_7959    195   f   1974
6     c_8026    193   m   1978
7     c_2595    193   m   1974
8     c_3725    190   f   1980
10    c_8392    189   f   1978


In [35]:
#On peut constater ci-dessus que les 4 premiers clients ont un montant d'achat bien supérieur au reste des clients. 
#Il est donc préférable de les enlever car cela fausserait les futures analyses.
mask = transactions.loc[(transactions['client_id'] == 'c_1609') | (transactions['client_id'] =='c_6714') | (transactions['client_id'] =='c_3454') | (transactions['client_id'] =='c_4958')]
top_clients = mask.index.tolist()
transactions = transactions.drop(top_clients)

In [36]:
df = pd.merge(transactions, customers, on='client_id')
df = pd.merge(df, products, on = 'id_prod')
df['count'] = 1
df = df.groupby('client_id').sum().reset_index()
df = df.sort_values('count', ascending=False)
df = df[['client_id', 'count']]
df = pd.merge(df, customers, on='client_id')
df = df.sort_values('count', ascending=False)
top_10 = df.iloc[0:10]
print(top_10)

  client_id  count sex  birth
0    c_2140    195   f   1977
1    c_7959    195   f   1974
2    c_8026    193   m   1978
3    c_2595    193   m   1974
4    c_3725    190   f   1980
5    c_8392    189   f   1978
6    c_7421    189   m   1978
7    c_3263    188   f   1985
8    c_2077    188   f   1984
9    c_8556    187   m   1976


5 - EXPORTATION DATAFRAMES

In [37]:
transactions.to_csv('transactions2.csv')
products.to_csv('products2.csv')
customers.to_csv('customers2.csv')